In [2]:
import re
import copy
import os
import shutil
import nltk
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from collections import Counter
import snowballstemmer as sb
import datetime

In [30]:
class Prompt:
    p = re.compile('\(.*\)')
    
    def __init__(self, n, t, c, d=None, is_group=False):
        if is_group:
            self.number = n
            self.interval = t
            self.content = c
            self.descriptions = [desc for desc in d if desc]
        else:
            self.number = n
            # "00:00:60.200 --> 00:00:80.50"
            # TODO parse time maybe?
            self.interval = [l.strip() for l in t.split('-->')]
            
            self.descriptions = []
            c_updated = []
            for l in c:
                # Speech... Storing this info might be useful...
                # Ex: "(Brock)"
                l = l[1:] if l.startswith('-') else l
                m = self.p.search(l)
                while m:
                    self.descriptions.append(m.group()[1:-1])
                    l = (l[:m.span()[0]] + l[m.span()[1]:]).strip()
                    m = self.p.search(l)
                c_updated.append(l)
            
            self.descriptions = None if len(self.descriptions) == 0 else self.descriptions
            self.content = ' '.join(c_updated)
        self.is_group = is_group
    
    def extend_content(self, other_content):
        return '{} {}'.format(
            self.content[:-3] if self.content.endswith('...') else self.content,
            other_content[3:] if other_content.startswith('...') else other_content
        )

    def __add__(self, other):
        combined = None
        if self.is_group:
            combined = copy.copy(self)

            combined.number.append(other.number)
            combined.interval.append(other.interval)
            combined.descriptions.append(other.descriptions)
            combined.content = self.extend_content(other.content)
        else:
            combined = Prompt([self.number, other.number],
                              [self.interval, other.interval],
                              self.extend_content(other.content),
                              [self.descriptions, other.descriptions],
                              True)
        return combined

    def __repr__(self):
        return '{}: {} || {}\n{}\n'.format(self.number, self.descriptions,
                                           self.interval, self.content)

## Parsing srt file

In [33]:
prompts = []
with open('substitle.srt', 'r', encoding='iso8859_9') as f:
    chunks = f.read().split('\n\n')
    for chunk in chunks:
        if chunk == '':
            continue
        lines = chunk.split('\n')
        prompts.append(Prompt(lines[0], lines[1], lines[2:]))

In [34]:
for prompt in prompts:
    print(prompt)
    
print(' '.join([p.content for p in prompts]))

1: None || ['00:00:06,095', '00:00:14,087']
Bu Filmin Betimlemesi Sesli Betimleme Derneği Tarafından Yapılmıştır.

2: None || ['00:00:04,572', '00:00:05,592']
www.sebeder.org

3: ['Müzik - Jenerik'] || ['00:00:06,095', '00:00:14,087']


4: ['Sessizlik'] || ['00:00:21,849', '00:00:25,739']


5: ['Müzik - Duygusal'] || ['00:00:26,340', '00:00:34,340']


6: ['Müzik - Duygusal'] || ['00:00:43,983', '00:00:51,976']


7: ['Müzik devam ediyor'] || ['00:01:02,030', '00:01:10,029']


8: ['Müzik devam ediyor'] || ['00:01:19,989', '00:01:27,985']


9: ['Müzik - Duygusal', 'Su sesi'] || ['00:01:28,563', '00:01:34,976']
 

10: ['Müzik - Duygusal'] || ['00:01:35,507', '00:01:43,499']


11: ['Anlaşılmayan telsiz sesi'] || ['00:01:50,834', '00:01:56,443']


12: ['Erkek sesi'] || ['00:01:56,688', '00:01:58,804']
On üç metre. Görüyor olmanız gerek.

13: ['Anlaşılmayan telsiz sesi'] || ['00:01:59,295', '00:02:03,091']


14: ['Su sesi'] || ['00:02:03,420', '00:02:06,264']


15: ['Müzik'] || ['00:02:06,959

1666: None || ['01:32:01,135', '01:32:03,856']
Polislere benziyor. Eskiden polismiş.

1667: ['Jack', 'Rose'] || ['01:32:04,503', '01:32:06,801']
Ahh lanet olsun! Ah hayır!

1668: ['Jack'] || ['01:32:07,957', '01:32:08,824']
Hadi.

1669: ['Rose'] || ['01:32:10,082', '01:32:12,082']
Koş. Hayır gel buraya.

1670: ['Kapı açılma sesi', 'Rose'] || ['01:32:12,708', '01:32:13,926']
 Ayy!

1671: ['Kapı kapanma sesi'] || ['01:32:14,465', '01:32:15,520']


1672: ['Uğultu sesi'] || ['01:32:17,403', '01:32:20,598']


1673: None || ['01:32:20,981', '01:32:22,918']
Şimdi ne yapacağız? Ne?

1674: ['Erkek sesi'] || ['01:32:23,676', '01:32:27,402']
Hadi, tembellik yok çocuklar, devam edin.

1675: ['Mekanik sesler'] || ['01:32:27,981', '01:32:30,933']


1676: ['Erkek sesi'] || ['01:32:31,191', '01:32:33,152']
Bir dakika siz burada ne arıyorsunuz?

1677: ['Rose'] || ['01:32:33,825', '01:32:35,652']
Durun burası tehlikelidir. Koş Jack!

1678: None || ['01:32:35,786', '01:32:36,786']
Durun dedim.

1679: ['M


2916: None || ['02:45:10,688', '02:45:11,717']
Jack!

2917: ['Çığlık sesleri'] || ['02:45:12,092', '02:45:16,248']


2918: None || ['02:45:16,632', '02:45:17,632']
Jack!

2919: None || ['02:45:18,147', '02:45:19,272']
Jack!

2920: ['Rose'] || ['02:45:20,748', '02:45:21,749']
Jack!

2921: ['Rose'] || ['02:45:22,335', '02:45:23,569']
Jack!

2922: ['Kargaşa sesleri'] || ['02:45:24,013', '02:45:28,659']


2923: ['Çığlık sesleri', 'Su sesi'] || ['02:45:29,056', '02:45:35,550']
 

2924: ['Nefes sesi', 'Su sesi'] || ['02:45:35,863', '02:45:38,050']
 

2925: None || ['02:45:38,339', '02:45:40,019']
Jack! Jack!

2926: ['Nefes sesi'] || ['02:45:40,410', '02:45:42,120']


2927: ['Uğultu sesi'] || ['02:45:42,472', '02:45:44,027']


2928: ['Nefes sesi'] || ['02:45:44,433', '02:45:47,253']


2929: None || ['02:45:47,668', '02:45:48,668']
Jack!

2930: None || ['02:45:49,441', '02:45:50,441']
Rose!

2931: None || ['02:45:51,020', '02:45:52,911']
Bırak onu bırak!

2932: ['Yumruk vurma sesi'] || ['02:4

## Merging prompts

In [35]:
combined_prompts = []
idx = 0
while idx != len(prompts):
    combined = None
    prompt = prompts[idx]
    while prompt.content.endswith('...'):
        idx += 1
        next_prompt = prompts[idx]
        combined = (combined + next_prompt) if combined else (prompt + next_prompt)
        prompt = next_prompt

    if combined is None:
        combined_prompts.append(prompt)
    else:
        combined_prompts.append(combined)
    idx += 1

In [8]:
#for prompt in combined_prompts[:20]:
#    print(prompt)

for p in combined_prompts:
    c = p.content.strip()
    if c != '':
        for sent in nltk.sent_tokenize(c):
            print(' '.join(nltk.word_tokenize(sent)))

Bu Filmin Betimlemesi Sesli Betimleme Derneği Tarafından Yapılmıştır .
www.sebeder.org
On üç metre .
Görüyor olmanız gerek .
Tamam .
Yavaşça burnunun üstüne çıkıyor
Tamam Mir- 2 , burnun üstüne çıkıyoruz arkamızdan gelin .
Tamam , devam et .
Çekiyoruz , güzel onun karanlıkta , bir hayalet gemi gibi karşımıza çıkması her seferinde , beni hâlâ etkiliyor .
Müthiş geminin , hüzünlü kalıntılarının burada 15 Nisan 1912 gecesi , saat 02:30'daki uzun düşüşünden sonra oturduğu yerde görmek beni gerçekten üzüyor .
Amma palavra sıkıyorsun patron .
Altıncı dalışımız .
Yine Titanic'in güvertesindeyiz .
Yaklaşık , dört kilometre derinlikteyiz .
Tam olarak 3.821 metre dışarıdaki basınç , santimetrekare başına , beş milibar .
Bu pencereler 23 santim kalınlığında ama patlarlarsa hepimiz yaşama veda ederiz .
Tamam .
Bu kadar saçmalık yeter .
Aracı dünkü gibi , üst güvertenin tepesine indirin .
Tamam .
Dinleyim Mir 2 , dünkü gibi geminin üzerine iniyoruz Dunkını dışarı çıkartın .
Açıldı .
Dunkın dolaşma

Sağ ol Sparks .
Oo telaşlanmayın canım .
Yılın bu zamanında , çok normaldir .
Hatta hızımızı arttırıyoruz .
Son kazanları da devreye soktuk .
Hadi oğlum şimdi sana gösterdiğim gibi ipi sar , evet .
Şimdi fırlat , çok güzel .
Nasıl oldu baba ?
Harika .
Güzel çok güzel , çok güzel .
Bay Andrews , kusura bakmayın ama kafamda bir hesap yaptım .
Az önce , filikaların alacağı yolcu sayısından bahsettiniz .
Ama galiba , gemideki her yolcuya yetecek kadar yer yok .
Öyle değil mi ?
Aslında , yarısına denk geliyor .
Gözünden hiçbir şey kaçmıyor , değil mi ?
Buraya , fazladan yer açmıştım .
Daha çok filika koymak istemiştim .
Tam buraya .
Ama bazıları , güvertelerin çok hantal görüneceğini düşündü .
O yüzden , kararımı kabul etmediler .
Batmayan bir gemide , bunlar bile fazlalık bence .
Rahat uyu Rose size iyi , sağlam ve güçlü bir gemi yaptım .
Bütün filikalar yeter .
Devam edelim .
Bundan sonra makina dairesine ineceğiz .
Gel .
Jack !
Bu imkansız .
Seninle görüşemem .
Seninle konuşmam gerek .
O

Yardımınıza ihtiyacım var !
Lütfen rica ediyorum .
Aşağıda bir adam kaldı lütfen .
Hanımefendi lütfen .
Hayır !
Paniğe kapılmanıza gerek yok .
Hayır paniğe kapılmıyorum .
Yanlış taraf gidiyorsunuz .
Gelin diyorum size Hanımefendi .
Bırak beni , bıraksana ya !
Lütfen rica ediyorum .
Bıraksana !
Niçin korkuyorsunuz lütfen !
Ahh !
Canın cehenneme !
Filikaları indirin .
İndirin , yavaş yavaş indirin .
Filikalar nerede ?
Filikaları indirin .
Tanrı 'm !
Ahh !
Jack !
Rose !
Bu işe yarar mı ?
Birazdan öğreniriz .
Hadi .
Dur , dur , dur , dur !
Şurada bir kaç alıştırma yap .
Güzel !
Şimdi aynı yere tekrar vurmaya çalış Rose .
Yapabilirsin !
Tamam .
Bu kadar alıştırma yeter .
Hadi Rose yapabilirsin .
Dinle !
Çok sert ve hızlı vurman gerekiyor tamam mı ?
Hadi .
Dur !
Ellerini biraz aç .
Evet güzel .
Nasıl ?
Evet .
Dinle Rose , sana güveniyorum .
Hadi !
Ahh !
Başardın !
Başardın !
Hadi gidelim hadi .
Ahh Tanrı 'm çok soğuk , çok soğuk !
Lanet olsun !
Lanet olsun !
Lanet olsun !
Çıkış yolu burasıyd

## Hasim Morphological Analysis

In [3]:
freq_hasim = Counter()

p_anl = re.compile('\[.*\]')
with open('titanic.disamb.txt', 'r') as f:
    for idx, line in enumerate(f.readlines()):
        if line[:3] == '<S>' or line[:4] == '</S>':
            continue
        
        raw, anl = line.strip().split()[:2]
        #print(raw)
        #print(anl)
        m = p_anl.search(anl)
        #print(m)
        if m:
            if m.group() != '[Punc]':
                freq_hasim[anl[:m.span()[0]]] += 1
        else:
            print('Warning in line {}: {}'.format(idx, line))
            
print(freq_hasim)
print(freq_hasim)

Counter({'bir': 215, 'ol': 201, 'hadi': 189, 'bu': 176, 'ben': 166, 'et': 139, 'o': 102, 'Rose': 99, 'Jack': 99, 'de': 95, 've': 94, 'ne': 89, 'mi': 78, 'çok': 77, 'yap': 74, 'var': 70, 'Tamam': 69, 'evet': 69, 'mı': 68, 'git': 67, 'gel': 67, 'sen': 66, 'bak': 66, 'efendi': 66, 'ama': 65, 'geri': 65, 'gemi': 63, 'bura': 62, 'lütfen': 62, 'hayır': 60, 'çabuk': 57, 'şey': 56, 'dur': 55, 'bil': 54, 'iste': 53, 'değil': 52, 'çek': 50, 'yok': 50, 'bay': 50, 'gör': 49, 'mu': 48, 'da': 47, 'çık': 46, 'iyi': 46, 'siz': 41, 'gibi': 40, '``': 40, 'şu': 39, 'Tanrı': 39, 'tam': 38, 'taraf': 38, 'filika': 38, 'daha': 37, 'kadar': 36, 'kadın': 36, 'çocuk': 35, 'için': 35, 'biz': 35, 'in': 34, 'yavaş': 34, 'al': 33, 'AA': 33, "''": 33, 'güzel': 32, 'her': 32, 'ver': 32, 'bul': 31, "'m": 30, 'can': 30, 'şimdi': 29, 'dön': 29, 'bin': 29, 'sadece': 28, 'gerek': 27, 'san': 26, 'kal': 26, 'Ahh': 26, 'yardım': 26, 'yer': 25, 'el': 25, 'hiç': 25, 'on': 24, 'gelin': 24, 'su': 24, 'yüz': 24, 'tabii': 24, 'hem

In [37]:
def sum_time(time_list):
    sum = datetime.timedelta()
    for i in time_list:
        sum += convert_delta(i)
    return sum
    
def convert_delta(time):
    try:
        (h, m, s) = time.split(':')
    except:
        (m, s) = time.split(':')
        h = 0
    d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(float(s)))
    return d

def sub_time(time1,time2):
    d = convert_delta(time1) - convert_delta(time2)
    print(str(d))

In [38]:
video_dif_list = ['01:28','28:59','55:51','1:24:31','1:45:47','2:11:36','2:38:32']
video_len = ['27:33','26:55','28:43','21:19','25:51','27:00','29:11']
video_list = ['MVI_8191.MOV','MVI_8192.MOV','MVI_8193.MOV','MVI_8194.MOV','MVI_8195.MOV','MVI_8196.MOV','MVI_8197.MOV']
video_list = ['titanik/titanik işaret dili/' + video for video in video_list]
for i in range(1,len(video_dif_list)):
    sub_time(video_dif_list[i],video_dif_list[i-1])
    print((convert_delta(video_len[i-1])))
    print('----------------------------------')
video_dif_list_delta = []
for t in video_dif_list:
    video_dif_list_delta.append(convert_delta(t))    
print(sum_time(video_len))

0:27:31
0:27:33
----------------------------------
0:26:52
0:26:55
----------------------------------
0:28:40
0:28:43
----------------------------------
0:21:16
0:21:19
----------------------------------
0:25:49
0:25:51
----------------------------------
0:26:56
0:27:00
----------------------------------
3:06:32


In [39]:
def select_video(start):
    video_index = 0
    for i in range(len(video_dif_list_delta)):
        #print(start,video_dif_list_delta[i])
        if start > video_dif_list_delta[i]:
            video_index = i
        else:
            break
    return video_list[video_index],start - video_dif_list_delta[video_index]

In [40]:
def find_places(word):
    places = []
    for prompt in prompts:
        if prompt.content is not None:
            if word in prompt.content:
                start = convert_delta(prompt.interval[0].replace(',','.'))
                end = convert_delta(prompt.interval[1].replace(',','.'))
                duration = end - start
                vd,vd_start = select_video(start=start)
                sb = [vd,vd_start.total_seconds(),vd_start.total_seconds()+duration.total_seconds()]
                print(len(places),vd,vd_start,duration)
                places.append(sb)
    return places

In [43]:
word = 'filika'
try:
    shutil.rmtree('titanik_local/'+ word)
    print('Deleted')
    os.mkdir(os.path.join('titanik_local/', word))
    print(word + ' created.')
except:
    os.mkdir(os.path.join( word))
    print(word + ' created.')


filika created.


In [44]:
P = find_places(word)

0 titanik/titanik işaret dili/MVI_8193.MOV 0:20:46 0:00:03
1 titanik/titanik işaret dili/MVI_8193.MOV 0:21:01 0:00:03
2 titanik/titanik işaret dili/MVI_8193.MOV 0:21:20 0:00:01
3 titanik/titanik işaret dili/MVI_8195.MOV 0:05:49 0:00:03
4 titanik/titanik işaret dili/MVI_8195.MOV 0:07:02 0:00:03
5 titanik/titanik işaret dili/MVI_8195.MOV 0:08:21 0:00:03
6 titanik/titanik işaret dili/MVI_8195.MOV 0:08:34 0:00:04
7 titanik/titanik işaret dili/MVI_8195.MOV 0:10:46 0:00:02
8 titanik/titanik işaret dili/MVI_8195.MOV 0:10:57 0:00:03
9 titanik/titanik işaret dili/MVI_8195.MOV 0:11:27 0:00:02
10 titanik/titanik işaret dili/MVI_8195.MOV 0:12:36 0:00:03
11 titanik/titanik işaret dili/MVI_8195.MOV 0:12:50 0:00:02
12 titanik/titanik işaret dili/MVI_8195.MOV 0:21:52 0:00:03
13 titanik/titanik işaret dili/MVI_8195.MOV 0:22:28 0:00:02
14 titanik/titanik işaret dili/MVI_8195.MOV 0:22:32 0:00:03
15 titanik/titanik işaret dili/MVI_8195.MOV 0:22:40 0:00:01
16 titanik/titanik işaret dili/MVI_8195.MOV 0:22:4

In [45]:
for i,vd in enumerate(P):
    print(vd)
    ffmpeg_extract_subclip(vd[0], vd[1]+1,vd[2]+1.8, targetname= 'titanik_local/'+word+"/"+"testnewton"+str(i) + ".mp4")
    print(vd[1],vd[2])

['titanik/titanik işaret dili/MVI_8193.MOV', 1246.0, 1249.0]

[MoviePy] Running:
>>> ffmpeg -y -i titanik/titanik işaret dili/MVI_8193.MOV -ss 1247.00 -t 3.80 -vcodec copy -acodec copy titanik_local/filika/testnewton0.mp4
[MoviePy] This command returned an error !

OSError: ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with Apple LLVM version 9.1.0 (clang-902.0.39.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.0.2 --enable-shared --enable-pthreads --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-gpl --enable-libmp3lame --enable-libx264 --enable-libxvid --enable-opencl --enable-videotoolbox --disable-lzma
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
titanik/titanik işaret dili/MVI_8193.MOV: No such file or directory


In [7]:
dicto = {}
p_anl = re.compile('\[.*\]')
with open('titanic.disamb.txt', 'r') as f:
    for idx, line in enumerate(f.readlines()):
        if line[:3] == '<S>' or line[:4] == '</S>':
            continue
        
        raw, anl = line.strip().split()[:2]
        #print(anl)
        m = p_anl.search(anl)
        #print(m)
        if m:
            if m.group() != '[Punc]':
                dicto[raw] = anl[:m.span()[0]]
        else:
            print('Warning in line {}: {}'.format(idx, line))

In [13]:
a = sb.TurkishStemmer()

In [28]:
a.stemWord(word='yapiyor')

'yapiyor'

In [46]:
import pipeline_caller


In [47]:
caller = pipeline_caller.PipelineCaller()



In [ ]:
caller()